In [ ]:
##################################
###   merge GIS and SAP data   ###
##################################

input:
* GIS forest map for a given year  (shape)
* SAP forest data for a given year (tsv)

output:
* merged forest map with SAP data   (shape)

In [1]:
import sys
import geopandas
import pandas as pd
import numpy as np
#from pyproj import CRS
try:
    from osgeo import ogr, osr, gdal
except:
    sys.exit('ERROR: cannot find GDAL/OGR modules')

### change coordinate system (CRS)

In [2]:
# read data
#wo_geo = geopandas.read_file('/home/philipp/Data/change_detection/GIS_data/wo_2016_2020.gdb', layer='Stichtag_20160101')

# define crs of orthophotos
#new_crs = CRS.from_user_input('PROJCS["Austria_Lambert",GEOGCS["GCS_BESSEL_AUT",DATUM["D_BESSEL_AUT",SPHEROID["Bessel_1841",6377397.155,299.1528128,AUTHORITY["EPSG","7004"]]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",47.5],PARAMETER["central_meridian",13.333333333],PARAMETER["standard_parallel_1",46],PARAMETER["standard_parallel_2",49],PARAMETER["false_easting",400000],PARAMETER["false_northing",400000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]')
# reproject to new crs
#wo_geo = wo_geo.to_crs(new_crs)

# flight year join by location

# save to file
#wo_geo.to_file("/home/philipp/Data/change_detection/GIS_data/gis_wo_2016.shp")

### read data - GIS_tax

In [3]:
wo_geo = geopandas.read_file('/home/philipp/Data/change_detection/GIS_data/wo_2016.shp')

In [4]:
wo_geo.crs

<Derived Projected CRS: EPSG:31287>
Name: MGI / Austria Lambert
Axis Info [cartesian]:
- X[north]: Northing (metre)
- Y[east]: Easting (metre)
Area of Use:
- name: Austria.
- bounds: (9.53, 46.4, 17.17, 49.02)
Coordinate Operation:
- name: Austria Lambert
- method: Lambert Conic Conformal (2SP)
Datum: Militar-Geographische Institut
- Ellipsoid: Bessel 1841
- Prime Meridian: Greenwich

In [5]:
# create unique ID WO
wo_geo['WO'] = wo_geo['FORSTBETRI'].astype(str) + \
wo_geo['REVIER_NR'].astype(str).str.zfill(2) + \
wo_geo['ABTEILUNG'].astype(str).str.zfill(3) + \
wo_geo['UNTERABTEI'] + \
wo_geo['TEILFLAECH'].astype(str)

# set data type of FLUGJAHR to int
wo_geo['FLUGJAHR'] = wo_geo['FLUGJAHR'].fillna(0)
wo_geo['FLUGJAHR'] = wo_geo['FLUGJAHR'].astype(int)

#rename columns
wo_geo.columns = ['obj_id', 'fb', 'fr', 'abt', 'uabt',
       'teilfl', 'color_code', 'link_id', 'id', 'admin', 'creation',
       'timeliness', 'length', 'area', 'fly_date', 'year_fly', 'geometry', 'WO']

# drop unnecessary columns
wo_geo = wo_geo.drop(['obj_id', 'color_code', 'link_id', 'id', 'creation', 'timeliness'], axis=1)

In [6]:
wo_geo.head()

,fb,fr,abt,uabt,teilfl,admin,length,area,fly_date,year_fly,geometry,WO
0,171,1,-1,0,1,529,195.920109,1553.41910,2015,2015-08-30,"POLYGON ((616346.810 493670.595, 616342.890 49...",171010-101
1,171,1,-1,0,1,529,1125.586021,35217.65235,2015,2015-08-30,"POLYGON ((615325.040 492902.555, 615263.980 49...",171010-101
2,171,1,-1,0,1,529,946.503132,24184.69905,2015,2015-08-30,"POLYGON ((608107.790 487993.105, 608108.120 48...",171010-101
3,171,1,-1,0,1,529,319.914046,5266.03840,2015,2015-08-30,"POLYGON ((614628.190 492540.735, 614612.100 49...",171010-101
4,171,1,506,3,1,529,2402.546849,11624.65550,2015,2015-08-30,"POLYGON ((617139.640 493799.045, 617151.260 49...",1710150631


In [7]:
wo_geo.to_file("/home/philipp/Data/change_detection/GIS_data/wo_2016.shp")

### load SAP tax data

In [3]:
# set year
year = 2016
# set sap tax path directory
path_sap_tax_dir = '/home/philipp/Data/GIS/SAP_tax'

path_sap_info = path_sap_tax_dir + '/edin_meta_data.xlsx'
sap_info = pd.read_excel(path_sap_info)#, engine='openpyxl')

sap_info.head()

,FB,FR,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,171,1,1208,1208,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1356,1356
1,171,2,1208,1208,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1356,1356
2,171,3,1208,1208,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1356,1356
3,171,4,1208,1208,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1356,1356
4,171,5,1208,1208,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1356,1356


In [4]:
# array with all TOs in a given year
tos = np.sort(sap_info.loc[:,year].unique())
tos

array([1030, 1031, 1041, 1042, 1043, 1044, 1045, 1048, 1049, 1050, 1051,
       1052, 1053, 1055, 1056, 1057, 1061, 1065, 1066, 1067, 1069, 1070,
       1071, 1073, 1074, 1075, 1079, 1081, 1082, 1083, 1085, 1086, 1088,
       1092, 1093, 1100, 1140, 1144, 1151, 1156, 1194, 1196, 1197, 1250,
       1283])

In [5]:
dict_sap_tax = {'Merkmalausprägung': np.uint8, \
        'AuswKatTyp': np.uint8, \
        'Teiloperats-ID': np.uint16, \
        'Forstbetrieb': np.uint8, \
        'Debitor': int, \
        'TO-Bezeichnung': str, \
        'Status': np.uint8, \
        'Beg. Laufzeit': str, \
        'Ende Laufzeit': str, \
        'Operat-ID': np.uint16, \
        'vorgeschl. Hiebssatz': int, \
        'Verantwortlicher': str, \
        'Erfassungsstatus': str, \
        'Migriert?': str, \
        'GUID': str, \
        'Forstbetrieb.1': np.uint8, \
        'Teiloperats-ID.1': np.uint16, \
        'Forstrevier': np.uint8, \
        'Abteilung': np.uint16, \
        'Unterabteil.': str, \
        'Teilfl.': np.uint8, 
        'Debitor.1': int, \
        'Bearbeitungsblock': np.uint8, \
        'WE-Typ': str, \
        'Betriebsklasse': np.uint16, \
        'Umtriebszeit': np.uint8, \
        'Nebengrund Art': np.uint8, \
        'Ertragssituation':str,
        'Bewirtschaftungsform': str, \
        'Schutzwaldkategorie': str, \
        'Fläche in HA': np.float64, \
        'Seehöhe': np.uint16, \
        'Exposition': str, \
        'Neigung': np.uint8, \
        'Standorteinheit': np.uint8, \
        'Vegetationstyp': str,
        'Waldtyp': str, \
        'Wuchsgebiet': str, \
        'Überh. Laubholz': np.uint16, \
        'Überh. Nadelhz.': np.uint16, \
        'fr. Schälschade': str, \
        'Verbissgrad': np.uint8, \
        'SchutzwaldProjNr': str, \
        'Schlussgrad': np.uint8, \
        'Stabilität': np.uint8, \
        'VJ Bedingung': np.uint8, \
        'VJ Situation': np.uint8, \
        'Erreichbark. des BZ': str, \
        'Selektiver Verbiss': np.uint8, \
        'Erfassungsstatus.1': str, \
        'Storno': str, \
        'Angelegt von': str, \
        'Angelegt am': str, \
        'Uhrzeit': str, \
        'Geändert von': str, \
        'Geändert am': str, \
        'Uhrzeit.1': str, \
        'Waldort':str, \
        'GUID.1':str, \
        'GUID.2':str, \
        'Forstbetrieb.2': np.uint8, \
        'Teiloperats-ID.2': np.uint16, \
        'Forstrevier.1': np.uint8, \
        'Abteilung.1': np.uint16, \
        'Unterabteil..1': str, \
        'Teilfl..1': np.uint8, \
        'Best.-Schicht': np.uint8, \
        'Debitor.2': int, \
        'Schichtanteil': np.uint8, \
        'Schichtalter': np.uint16, \
        'S-Best.grad': np.float16, \
        'Erfassungsstatus.2': str, \
        'Storno.1':str, \
        'GUID.3': str, \
        'GUID.4': str, \
        'Forstbetrieb.3': np.uint8, \
        'Teiloperats-ID.3': np.uint16, \
        'Forstrevier.2': np.uint8, \
        'Abteilung.2': np.uint16, \
        'Unterabteil..2': str, \
        'Teilfl..2': np.uint8, \
        'Best.-Schicht.1': np.uint8, \
        'Baumart': str, \
        'Debitor.3': int, \
        'Baumartenanteil': np.uint8, \
        'BaumartenBestockgrad': np.float16, \
        'Schälgrad': np.uint8,
        'Ertragsklasse': np.float16, \
        'Vorrat / ha': np.float32, \
        'Laubholzvorrat / ha': np.float32, \
        'Nadelholzvorrat / ha': np.float32, \
        'Vorrat am Ort': np.float32, \
        'Laubholzvorrat Ort': np.float32, \
        'Nadelholzvorrat Ort': np.float32, \
        'GSOLL / HA': np.float32, \
        'GIST / HA': np.float32, \
        'lauf. Zuwachs / HA': np.float16, \
        'DGZU / HA': np.float16, \
        'HDZ': np.float16, \
        'LGZ': np.float16, \
        'Erfassungsstatus.3': str, \
        'Storno.2': str, \
        'GUID.5': str, \
        'GUID.6': str, \
        'Forstbetrieb.4': np.uint8, \
        'Teiloperats-ID.4': np.uint16, \
        'Forstrevier.3': np.uint8, \
        'Abteilung.3': np.uint16, \
        'Unterabteil..3': str, \
        'Teilfl..3': np.uint8, \
        'Best.-Schicht.2': np.uint8, \
        'Nutzungsnummer': np.uint8, \
        'Maßnahmenart': str, \
        'Massnahme geplant': str, \
        'Massnahmengruppe': str, \
        'Angriffsfläche': np.float32, \
        'Nutzung LH': np.uint16, \
        'Nutzung NH': np.uint16, \
        'Nutzungssumme': np.uint16, \
        'Nutzdringlichkeit': np.uint8, \
        'Bewpfl.': np.uint8, \
        'Zeitpunkt': np.uint8, \
        'Rückungsart': np.uint8, \
        'Schlägerungsart': np.uint8, \
        'Erfassungsstatus.4': str, \
        'Storno.3': str, \
        'Nutztext': str, \
        'Alter der 1. Schicht': np.uint16, \
        'TAX: Altersklasse': str, \
        'Repr. Fläche Schicht': np.float32, \
        'Produktionskategorie': str, \
        'Geschäftsjahr': np.uint8, \
        'Abmaßbeleg': str, \
        'Maßnahme': str, \
        'Geschäftsfeld': str, \
        'Bezeichnung': str, \
        'Pflanzen Ist': str, \
        'Baumarten Ist': str, \
        'Repr. Fläche Baumart': np.float32, \
        'Ertragstafelnummer': np.uint8, \
        'Ertragstafelbezeich': str, \
        'Anmerkung': str, \
        'Zeile1': str, \
        'Zeile2': str, \
        'Zeile3': str, \
        'Zeile4': str, \
        'Zeile5': str, \
        'Zeile6': str, \
        'Zeile7': str, \
        'Zeile8': str, \
        'Bestockungsziel': str, \
        'Flächenanteil': np.float32}

In [6]:
def get_data(path, tos):

    wo_sap_list = []

    for to in tos:
        # get fb
        fb = sap_info.loc[sap_info[year] == to, 'FB'].unique()[0]
        # create path to file
        path_sap_tax_file = path + '/' + str(fb) + '/TO_' + str(to) + '.XLS'

        print(path_sap_tax_file)

        # read dat from file
        wo_sap_list.append(pd.read_csv(path_sap_tax_file, 
                                       sep='\t',
                                       encoding = "ISO-8859-1", 
                                       decimal=',', 
                                       error_bad_lines=False))
        # create unique ID WO
        #wo_sap['WO'] = wo_sap['Forstbetrieb'].astype(str) + \
        #wo_sap['Forstrevier'].astype(str) + \
        #wo_sap['Abteilung'].astype(str) + \
        #wo_sap['Unterabteil.'] + \
        #wo_sap['Teilfl.'].astype(str)

        #wo_sap_list.append(wo_sap)
        
        # concatonate all dataframes
    wo_sap = pd.concat(wo_sap_list, axis=0, ignore_index=True)
    # fill no data with 0
    wo_sap = wo_sap.fillna(0)
    # set correct data type
    for col, dtype in dict_sap_tax.items():
        print(col, dtype)
        wo_sap[col] =  wo_sap[col].astype(dtype)
        
    # create unique ID WO
    wo_sap['WO'] = wo_sap['Forstbetrieb'].astype(str) + \
    wo_sap['Forstrevier'].astype(str).str.zfill(2) + \
    wo_sap['Abteilung'].astype(str).str.zfill(3) + \
    wo_sap['Unterabteil.'] + \
    wo_sap['Teilfl.'].astype(str)

    return wo_sap

In [7]:
# read data
wo_sap = get_data(path_sap_tax_dir, tos)

/home/philipp/Data/GIS/SAP_tax/171/TO_1030.XLS


/tmp/ipykernel_8306/1879515599.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  wo_sap = get_data(path_sap_tax_dir, tos)
/tmp/ipykernel_8306/1879515599.py:2: DtypeWarning: Columns (29,147,148,149) have mixed types.Specify dtype option on import or set low_memory=False.
  wo_sap = get_data(path_sap_tax_dir, tos)


/home/philipp/Data/GIS/SAP_tax/180/TO_1031.XLS
/home/philipp/Data/GIS/SAP_tax/180/TO_1041.XLS


/tmp/ipykernel_8306/1879515599.py:2: DtypeWarning: Columns (147) have mixed types.Specify dtype option on import or set low_memory=False.
  wo_sap = get_data(path_sap_tax_dir, tos)
/tmp/ipykernel_8306/1879515599.py:2: DtypeWarning: Columns (29,146,147,148,149) have mixed types.Specify dtype option on import or set low_memory=False.
  wo_sap = get_data(path_sap_tax_dir, tos)


/home/philipp/Data/GIS/SAP_tax/173/TO_1042.XLS


...

/tmp/ipykernel_8306/1879515599.py:2: DtypeWarning: Columns (40,148,149) have mixed types.Specify dtype option on import or set low_memory=False.
  wo_sap = get_data(path_sap_tax_dir, tos)
/tmp/ipykernel_8306/1879515599.py:2: DtypeWarning: Columns (146,147,148,149) have mixed types.Specify dtype option on import or set low_memory=False.
  wo_sap = get_data(path_sap_tax_dir, tos)


/home/philipp/Data/GIS/SAP_tax/179/TO_1196.XLS
/home/philipp/Data/GIS/SAP_tax/175/TO_1197.XLS
/home/philipp/Data/GIS/SAP_tax/182/TO_1250.XLS
/home/philipp/Data/GIS/SAP_tax/179/TO_1283.XLS


/tmp/ipykernel_8306/1879515599.py:2: DtypeWarning: Columns (148) have mixed types.Specify dtype option on import or set low_memory=False.
  wo_sap = get_data(path_sap_tax_dir, tos)


Merkmalausprägung <class 'numpy.uint8'>
AuswKatTyp <class 'numpy.uint8'>
Teiloperats-ID <class 'numpy.uint16'>
Forstbetrieb <class 'numpy.uint8'>
Debitor <class 'int'>
TO-Bezeichnung <class 'str'>
Status <class 'numpy.uint8'>
Beg. Laufzeit <class 'str'>
Ende Laufzeit <class 'str'>
Operat-ID <class 'numpy.uint16'>
vorgeschl. Hiebssatz <class 'int'>
Verantwortlicher <class 'str'>
Erfassungsstatus <class 'str'>
Migriert? <class 'str'>
GUID <class 'str'>
Forstbetrieb.1 <class 'numpy.uint8'>
Teiloperats-ID.1 <class 'numpy.uint16'>
Forstrevier <class 'numpy.uint8'>
Abteilung <class 'numpy.uint16'>
Unterabteil. <class 'str'>
Teilfl. <class 'numpy.uint8'>
Debitor.1 <class 'int'>
Bearbeitungsblock <class 'numpy.uint8'>
WE-Typ <class 'str'>
Betriebsklasse <class 'numpy.uint16'>
Umtriebszeit <class 'numpy.uint8'>
Nebengrund Art <class 'numpy.uint8'>
Ertragssituation <class 'str'>
Bewirtschaftungsform <class 'str'>
Schutzwaldkategorie <class 'str'>
Fläche in HA <class 'numpy.float64'>
Seehöhe <cla

/tmp/ipykernel_8306/2425343650.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  wo_sap['WO'] = wo_sap['Forstbetrieb'].astype(str) + \


In [9]:
wo_sap.head()

,Merkmalausprägung,AuswKatTyp,Teiloperats-ID,Forstbetrieb,Debitor,TO-Bezeichnung,Status,Beg. Laufzeit,Ende Laufzeit,Operat-ID,...,Zeile3,Zeile4,Zeile5,Zeile6,Zeile7,Zeile8,Bestockungsziel,Flächenanteil,Unnamed: 0,WO
0,0,0,1030,171,220442,1,2,01.01.2009,31.12.2019,111,...,diese hier älter (+5 J); ein Einzel-WW; im N-T...,vergrast und leicht verkrautet,MA BZ: 5LA 5BU,0,0,0,0,0.0,0.0,17101648B1
1,0,0,1030,171,220442,1,2,01.01.2009,31.12.2019,111,...,0,0,0,0,0,0,0,0.0,0.0,17101648B1
2,0,0,1030,171,220442,1,2,01.01.2009,31.12.2019,111,...,0,0,0,0,0,0,0,0.0,0.0,17101648B1
3,0,0,1030,171,220442,1,2,01.01.2009,31.12.2019,111,...,0,0,0,0,0,0,0,0.0,0.0,17101648B1
4,0,0,1030,171,220442,1,2,01.01.2009,31.12.2019,111,...,0,0,0,0,0,0,0,0.0,0.0,17101648B1


In [10]:
# stoe

# filter data
wo_sap_stoe = wo_sap.loc[wo_sap['Best.-Schicht'] == 0, ['WO', 'Forstbetrieb', 'Forstrevier', 'Abteilung', 
                                                        'Unterabteil.', 'Teilfl.', 'Beg. Laufzeit', 'Umtriebszeit', 
                                                        'Nebengrund Art', 'Ertragssituation', 'Bewirtschaftungsform', 
                                                        'Schutzwaldkategorie', 'Seehöhe', 'Exposition', 'Neigung', 
                                                        'Standorteinheit', 'Vegetationstyp', 'Wuchsgebiet']]

wo_sap_stoe.head()

,WO,Forstbetrieb,Forstrevier,Abteilung,Unterabteil.,Teilfl.,Beg. Laufzeit,Umtriebszeit,Nebengrund Art,Ertragssituation,Bewirtschaftungsform,Schutzwaldkategorie,Seehöhe,Exposition,Neigung,Standorteinheit,Vegetationstyp,Wuchsgebiet
0,17101648B1,171,1,648,B,1,01.01.2009,120,0,I,W,0,400,SO,27,87,BW,5.1
6,17101652C1,171,1,652,C,1,01.01.2009,120,0,I,W,0,400,SW,27,87,WW,5.1
12,17101736B4,171,1,736,B,4,01.01.2009,120,0,I,W,0,300,S,18,87,WW,5.1
21,17101658A1,171,1,658,A,1,01.01.2009,120,0,I,W,0,300,SW,18,87,WW,5.1
27,17101506D1,171,1,506,D,1,01.01.2009,120,0,I,W,0,300,NO,27,88,WW,5.1


In [11]:
# wood volume

# filter data
wo_sap_v = wo_sap.loc[wo_sap['Best.-Schicht.1'] > 0, ['WO', 'Vorrat / ha', 'Laubholzvorrat / ha', 
                                                      'Nadelholzvorrat / ha', 'Vorrat am Ort', 
                                                      'Laubholzvorrat Ort', 'Nadelholzvorrat Ort',]]

# group by WO (ID) and sum all values
wo_sap_v = wo_sap_v.groupby(['WO']).sum().reset_index()

wo_sap_v.head()

,WO,Vorrat / ha,Laubholzvorrat / ha,Nadelholzvorrat / ha,Vorrat am Ort,Laubholzvorrat Ort,Nadelholzvorrat Ort
0,17101506A0,331.200012,229.680008,101.519997,3083.469971,2138.320068,945.150024
1,17101506B1,236.699997,208.179993,28.520000,295.880005,260.230011,35.650002
2,17101506B2,13.000000,13.000000,0.000000,7.280000,7.280000,0.000000
3,17101506C1,175.000000,175.000000,0.000000,428.750000,428.750000,0.000000
4,17101506C2,89.000000,72.900002,16.100000,263.440002,215.779999,47.660000


In [12]:
# wood cuts

# filter data
wo_sap_ma = wo_sap.loc[wo_sap['Nutzungsnummer'] > 0, ['WO', 'Maßnahmenart', 'Massnahmengruppe', 'Angriffsfläche', 
                                                      'Nutzung LH', 'Nutzung NH', 'Nutzungssumme', 'Nutzdringlichkeit', 
                                                      'Bewpfl.', 'Zeitpunkt', 'Rückungsart', 'Schlägerungsart']]

wo_sap_ma.head()

,WO,Maßnahmenart,Massnahmengruppe,Angriffsfläche,Nutzung LH,Nutzung NH,Nutzungssumme,Nutzdringlichkeit,Bewpfl.,Zeitpunkt,Rückungsart,Schlägerungsart
5,17101648B1,RM,EN,4.0,1600,0,1600,2,3,2,30,1
10,17101652C1,DF,VN,3.0,100,0,100,2,1,2,30,1
11,17101652C1,ZE,EN,0.0,30,20,50,3,1,2,30,1
20,17101736B4,JP,WP,3.5,0,0,0,1,1,1,0,0
26,17101658A1,DF,VN,4.0,200,0,200,2,1,2,10,1


In [13]:
# filter just VN

wo_sap_maf = wo_sap_ma[wo_sap_ma['Massnahmengruppe'] == 'VN']
wo_sap_maf = wo_sap_maf[wo_sap_maf['Maßnahmenart'] != 'ZV']
wo_sap_maf = wo_sap_maf[wo_sap_maf['Maßnahmenart'] != 'UE']
wo_sap_maf = wo_sap_maf[wo_sap_maf['Maßnahmenart'] != 'LL']

print(wo_sap_maf['Maßnahmenart'].unique())

# group by WO (ID) and sum all values
wo_sap_maf = wo_sap_maf.groupby(['WO']).sum().reset_index()

wo_sap_maf['ma'] = 'DF'

['DF' 'DE' 'ND']


In [15]:
wo_sap_maf

,WO,Angriffsfläche,Nutzung LH,Nutzung NH,Nutzungssumme,Nutzdringlichkeit,Bewpfl.,Zeitpunkt,Rückungsart,Schlägerungsart
0,17101506A0,9.3,350,150,500,2.0,1.0,2.0,30.0,1.0
1,17101506C1,0.6,30,0,30,1.0,1.0,2.0,10.0,1.0
2,17101506C2,3.2,165,0,165,3.0,2.0,4.0,45.0,5.0
3,17101506D2,2.8,120,0,120,2.0,1.0,2.0,35.0,4.0
4,17101506G1,1.9,60,30,90,1.0,1.0,2.0,35.0,4.0
...,...,...,...,...,...,...,...,...,...,...
34087,18209366L1,2.5,0,120,120,1.0,1.0,1.0,35.0,4.0
34088,18209366L2,0.1,0,10,10,2.0,1.0,1.0,35.0,4.0
34089,18209366M2,1.9,0,80,80,1.0,1.0,1.0,35.0,4.0
34090,18209367H2,0.6,0,20,20,2.0,1.0,1.0,30.0,2.0


In [16]:
# species

# filter just necessary columns
wo_sap_ba = wo_sap.loc[wo_sap['Best.-Schicht.1'] > 0, 
                    ['WO','Best.-Schicht.1', 'Schichtanteil', 'Schichtalter', 
                     'S-Best.grad', 'Baumart','Baumartenanteil', 'BaumartenBestockgrad']]

wo_sap_sch = wo_sap.loc[(wo_sap['Best.-Schicht'] > 0) & (wo_sap['Best.-Schicht.1'] == 0), 
                    ['WO','Best.-Schicht.1', 'Schichtanteil', 'Schichtalter', 
                     'S-Best.grad', 'Baumart','Baumartenanteil', 'BaumartenBestockgrad']]

wos_unique = wo_sap_ba['WO'].unique()

In [17]:
lh_set = {'EI', 'EL', 'ES', 'EA', 'FA', 'FE', 'GB', 'WP', 'GE', 'AV', 'HB', 'HP', \
          'KB', 'LI', 'ME', 'PO', 'RO', 'RK', 'BU', 'RE', 'SW', 'ER', 'JN', 'SP', \
          'LS', 'SL', 'SG', 'SA', 'QR', 'ST', 'QP', 'TK', 'TB', 'UL', 'NU', 'WD', \
          'WO', 'LW', 'EZ', 'AH', 'AS', 'RU', 'BI', 'EE', 'EK', 'GP', 'KA', 'PA'}
nh_set = {'FZ', 'GK', 'AG', 'HT', 'JL', 'CJ', 'KK', 'KO', 'AN', 'FO', 'AB', 'CH', \
          'SF', 'SN', 'PU', 'KW', 'TH', 'TA', 'ZI', 'AZ', 'BK', 'AC', 'EB', 'OF', \
          'PM', 'TA'}
ba_set = {'LA', 'KI', 'SK', 'DG'}

def extract_species(one_wo):
    ba_dict = dict()
    ba_dict['WO'] = one_wo['WO'].iloc[0]
    ba_dict['alter'] = one_wo['Schichtalter'].iloc[0]
    ba_dict['BL'] = 0
    ba_dict['FI'] = 0
    ba_dict['LH'] = 0
    ba_dict['NH'] = 0

    # loop over all species
    for _, data in one_wo[['Baumart', 'Baumartenanteil']].transpose().items():

        if data[0] == 'BL':
            ba_dict['BL'] += data[1]
        elif (data[0] == 'FI') | (data[0] == 'TA'):
            ba_dict['FI'] += data[1]
        elif data[0] in ba_set:
            ba_dict[data[0]] = data[1]
        elif data[0] in lh_set:
            ba_dict['LH'] += data[1]
        elif data[0] in nh_set:
            ba_dict['NH'] += data[1]

    return ba_dict

In [18]:
def get_age_species(one_wo):
    if (one_wo['Best.-Schicht.1'].unique().size == 1):
        dic = extract_species(one_wo)
    elif(one_wo.loc[one_wo['Best.-Schicht.1'] == 1, 'S-Best.grad'].iloc[0] >= 0.5):
        dic = extract_species(one_wo.loc[one_wo['Best.-Schicht.1'] == 1])
    else:
        # wo id
        wo_id = one_wo.iloc[0,0]

        # age
        bg_max = one_wo['S-Best.grad'].max()
        s_ages = np.sort(one_wo['Schichtalter'].unique())[::-1]
        for s_age in s_ages:
            age = s_age
            s_bg = one_wo.loc[one_wo['Schichtalter']==s_age, 'S-Best.grad'].iloc[0]
            if (s_bg >= 0.5) | (s_bg == bg_max):
                age = s_age
                break
        
        # species
        one_wo_gb = one_wo.groupby(by=["Baumart"], as_index=False, sort=False).sum()
        one_wo_gb['Baumartenanteil'] = (one_wo_gb['BaumartenBestockgrad']/one_wo_gb['BaumartenBestockgrad'].sum() * 100).round(0).astype(int)
        one_wo_gb['WO'] = wo_id
        one_wo_gb['Schichtalter'] = age
        dic = extract_species(one_wo_gb)
    return dic

In [19]:
dd = dict()
for wo_unique in wos_unique:
    one_wo = wo_sap_ba.loc[(wo_sap_ba['WO'] == wo_unique)]
    dd[wo_unique] = get_age_species(one_wo)
print('finished')

finished


In [20]:
# transpose dataframe
wo_sap_ba = pd.DataFrame(dd).transpose()
# fill nan values with 0
wo_sap_ba = wo_sap_ba.fillna(0)

In [21]:
wo_sap_ba.head()

,WO,alter,BL,FI,LH,NH,LA,KI,DG,SK
17101648B1,17101648B1,140,0,0,100,0,0,0,0,0
17101652C1,17101652C1,100,0,10,90,0,0,0,0,0
17101736B4,17101736B4,5,0,0,60,0,40,0,0,0
17101658A1,17101658A1,60,0,0,100,0,0,0,0,0
17101506D1,17101506D1,135,0,0,100,0,0,0,0,0


### merge all SAP data

In [22]:
# merge SAP stoe & SAP tree species
wo_sap = pd.merge(wo_sap_stoe, wo_sap_ba, how='left', on='WO', sort=False,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

# merge SAP all & SAP volume
wo_sap = pd.merge(wo_sap, wo_sap_v, how='left', on='WO', sort=False,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

# merge SAP all & SAP planned wood cut
wo_sap = pd.merge(wo_sap, wo_sap_maf, how='left', on='WO', sort=False,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

wo_sap.head()

,WO,Forstbetrieb,Forstrevier,Abteilung,Unterabteil.,Teilfl.,Beg. Laufzeit,Umtriebszeit,Nebengrund Art,Ertragssituation,...,Nadelholzvorrat Ort,Angriffsfläche,Nutzung LH,Nutzung NH,Nutzungssumme,Nutzdringlichkeit,Bewpfl.,Zeitpunkt,Rückungsart,Schlägerungsart
0,17101648B1,171,1,648,B,1,01.01.2009,120,0,I,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17101652C1,171,1,652,C,1,01.01.2009,120,0,I,...,777.169983,3.0,100.0,0.0,100.0,2.0,1.0,2.0,30.0,1.0
2,17101736B4,171,1,736,B,4,01.01.2009,120,0,I,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,17101658A1,171,1,658,A,1,01.01.2009,120,0,I,...,0.000000,4.0,200.0,0.0,200.0,2.0,1.0,2.0,10.0,1.0
4,17101506D1,171,1,506,D,1,01.01.2009,120,0,I,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
wo_sap.columns

Index(['WO', 'Forstbetrieb', 'Forstrevier', 'Abteilung', 'Unterabteil.',
       'Teilfl.', 'Beg. Laufzeit', 'Umtriebszeit', 'Nebengrund Art',
       'Ertragssituation', 'Bewirtschaftungsform', 'Schutzwaldkategorie',
       'Seehöhe', 'Exposition', 'Neigung', 'Standorteinheit', 'Vegetationstyp',
       'Wuchsgebiet', 'alter', 'BL', 'FI', 'LH', 'NH', 'LA', 'KI', 'DG', 'SK',
       'Vorrat / ha', 'Laubholzvorrat / ha', 'Nadelholzvorrat / ha',
       'Vorrat am Ort', 'Laubholzvorrat Ort', 'Nadelholzvorrat Ort',
       'Angriffsfläche', 'Nutzung LH', 'Nutzung NH', 'Nutzungssumme',
       'Nutzdringlichkeit', 'Bewpfl.', 'Zeitpunkt', 'Rückungsart',
       'Schlägerungsart', 'ma'],
      dtype='object')

In [29]:
wo_sap.loc[wo_sap['Nutzdringlichkeit']>0,'ma'] = 'DF'

In [32]:
wo_sap.columns = ['WO', 'fb_sap', 'fr_sap', 'abt_sap', 'uabt_sap', 'teilfl_sap', \
                  'start_term', 'uz', 'non_forest_type', 'economy', 'ww_sw', 'sw_type', \
                  'sea_level', 'exp', 'slope', 'site_type', 'veg_type', 'growth_area', \
                  'age', 'BL', 'FI', 'LH', 'NH', 'LA', 'KI', 'DG', 'SK', \
                  'mass_ha', 'mass_ha_lh', 'mass_ha_nh', 'mass_tot', 'mass_tot_lh', \
                  'mass_tot_nh', 'cut_area', 'cut_lh', 'cut_nh', 'cut_sum', \
                  'dr', 'bp', 'zp', 'ru', 'sg', 'ma']

In [33]:
wo_sap['year_fe'] = wo_sap['start_term'].str[-4:].astype(int)

In [35]:
# write data to disk
wo_sap.to_csv('/home/philipp/Data/GIS/SAP_tax/SAP_2016.csv')

### Merge SAP & GIS data

In [ ]:
###########################
# prepare data for merge
###########################

In [ ]:
# read data from disk
wo_sap = pd.read_csv('/home/philipp/Data/GIS/SAP_tax/SAP_2016.csv', index_col='Unnamed: 0')
wo_geo = geopandas.read_file('/home/philipp/Data/change_detection/GIS_data/wo_2016.shp')

In [36]:
wo_sap.head()

,WO,fb_sap,fr_sap,abt_sap,uabt_sap,teilfl_sap,start_term,uz,non_forest_type,economy,...,cut_lh,cut_nh,cut_sum,dr,bp,zp,ru,sg,ma,year_fe
0,17101648B1,171,1,648,B,1,01.01.2009,120,0,I,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009
1,17101652C1,171,1,652,C,1,01.01.2009,120,0,I,...,100.0,0.0,100.0,2.0,1.0,2.0,30.0,1.0,DF,2009
2,17101736B4,171,1,736,B,4,01.01.2009,120,0,I,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009
3,17101658A1,171,1,658,A,1,01.01.2009,120,0,I,...,200.0,0.0,200.0,2.0,1.0,2.0,10.0,1.0,DF,2009
4,17101506D1,171,1,506,D,1,01.01.2009,120,0,I,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009


In [37]:
wo_geo.head()

,fb,fr,abt,uabt,teilfl,admin,length,area,fly_date,year_fly,WO,geometry
0,171,1,-1,0,1,529,195.920109,1553.41910,2015,2015-08-30,171010-101,"POLYGON ((616346.810 493670.595, 616342.890 49..."
1,171,1,-1,0,1,529,1125.586021,35217.65235,2015,2015-08-30,171010-101,"POLYGON ((615325.040 492902.555, 615263.980 49..."
2,171,1,-1,0,1,529,946.503132,24184.69905,2015,2015-08-30,171010-101,"POLYGON ((608107.790 487993.105, 608108.120 48..."
3,171,1,-1,0,1,529,319.914046,5266.03840,2015,2015-08-30,171010-101,"POLYGON ((614628.190 492540.735, 614612.100 49..."
4,171,1,506,3,1,529,2402.546849,11624.65550,2015,2015-08-30,1710150631,"POLYGON ((617139.640 493799.045, 617151.260 49..."


In [38]:
wo_sap = wo_sap.fillna(0)

In [39]:
wo_sap.columns

Index(['WO', 'fb_sap', 'fr_sap', 'abt_sap', 'uabt_sap', 'teilfl_sap',
       'start_term', 'uz', 'non_forest_type', 'economy', 'ww_sw', 'sw_type',
       'sea_level', 'exp', 'slope', 'site_type', 'veg_type', 'growth_area',
       'age', 'BL', 'FI', 'LH', 'NH', 'LA', 'KI', 'DG', 'SK', 'mass_ha',
       'mass_ha_lh', 'mass_ha_nh', 'mass_tot', 'mass_tot_lh', 'mass_tot_nh',
       'cut_area', 'cut_lh', 'cut_nh', 'cut_sum', 'dr', 'bp', 'zp', 'ru', 'sg',
       'ma', 'year_fe'],
      dtype='object')

In [40]:
wo_sap.iloc[:,:20].head()

,WO,fb_sap,fr_sap,abt_sap,uabt_sap,teilfl_sap,start_term,uz,non_forest_type,economy,ww_sw,sw_type,sea_level,exp,slope,site_type,veg_type,growth_area,age,BL
0,17101648B1,171,1,648,B,1,01.01.2009,120,0,I,W,0,400,SO,27,87,BW,5.1,140.0,0.0
1,17101652C1,171,1,652,C,1,01.01.2009,120,0,I,W,0,400,SW,27,87,WW,5.1,100.0,0.0
2,17101736B4,171,1,736,B,4,01.01.2009,120,0,I,W,0,300,S,18,87,WW,5.1,5.0,0.0
3,17101658A1,171,1,658,A,1,01.01.2009,120,0,I,W,0,300,SW,18,87,WW,5.1,60.0,0.0
4,17101506D1,171,1,506,D,1,01.01.2009,120,0,I,W,0,300,NO,27,88,WW,5.1,135.0,0.0


In [41]:
wo_sap['non_forest_type'] = wo_sap['non_forest_type'].astype(int)
wo_sap['sea_level'] = wo_sap['sea_level'].astype(int)
wo_sap['slope'] = wo_sap['slope'].astype(int)
wo_sap['site_type'] = wo_sap['site_type'].astype(int)
wo_sap['growth_area'] = wo_sap['growth_area'].astype(str)

wo_sap['age'] = wo_sap['age'].astype(int)
wo_sap['BL'] = wo_sap['BL'].astype(int)
wo_sap['FI'] = wo_sap['FI'].astype(int)
wo_sap['LH'] = wo_sap['LH'].astype(int)
wo_sap['NH'] = wo_sap['NH'].astype(int)
wo_sap['LA'] = wo_sap['LA'].astype(int)
wo_sap['KI'] = wo_sap['KI'].astype(int)
wo_sap['DG'] = wo_sap['DG'].astype(int)
wo_sap['SK'] = wo_sap['SK'].astype(int)

wo_sap['cut_lh'] = wo_sap['cut_lh'].astype(int)
wo_sap['cut_nh'] = wo_sap['cut_nh'].astype(int)
wo_sap['cut_sum'] = wo_sap['cut_sum'].astype(int)
wo_sap['dr'] = wo_sap['dr'].astype(int)
wo_sap['bp'] = wo_sap['bp'].astype(int)
wo_sap['zp'] = wo_sap['zp'].astype(int)
wo_sap['ru'] = wo_sap['ru'].astype(int)
wo_sap['sg'] = wo_sap['sg'].astype(int)
wo_sap['year_fe'] = wo_sap['year_fe'].astype(int)

In [42]:
# merge GIS & SAP
wo = wo_geo.merge(wo_sap, on='WO', how="left")

In [52]:
wo[['fly_date','year_fly']]

,fly_date,year_fly
0,2015-08-30,2015
1,2015-08-30,2015
2,2015-08-30,2015
3,2015-08-30,2015
4,2015-08-30,2015
...,...,...
354766,2015-08-30,2015
354767,2015-08-30,2015
354768,2015-08-30,2015
354769,2015-08-30,2015


In [53]:
wo[['fb', 'fr', 'abt', 'uabt', 'teilfl', 'admin', 'length', 'area',
       'fly_date', 'year_fly', 'WO', 'fb_sap', 'fr_sap', 'abt_sap',
       'uabt_sap', 'teilfl_sap', 'start_term', 'uz', 'non_forest_type',
       'economy', 'ww_sw', 'sw_type', 'sea_level', 'exp', 'slope', 'site_type',
       'veg_type', 'growth_area', 'age', 'BL', 'FI', 'LH', 'NH', 'LA', 'KI',
       'DG', 'SK', 'mass_ha', 'mass_ha_lh', 'mass_ha_nh', 'mass_tot',
       'mass_tot_lh', 'mass_tot_nh', 'cut_area', 'cut_lh', 'cut_nh', 'cut_sum',
       'dr', 'bp', 'zp', 'ru', 'sg', 'ma', 'year_fe']] = wo[['fb', 'fr', 'abt', 'uabt', 'teilfl', 'admin', 'length', 'area',
       'fly_date', 'year_fly', 'WO', 'fb_sap', 'fr_sap', 'abt_sap',
       'uabt_sap', 'teilfl_sap', 'start_term', 'uz', 'non_forest_type',
       'economy', 'ww_sw', 'sw_type', 'sea_level', 'exp', 'slope', 'site_type',
       'veg_type', 'growth_area', 'age', 'BL', 'FI', 'LH', 'NH', 'LA', 'KI',
       'DG', 'SK', 'mass_ha', 'mass_ha_lh', 'mass_ha_nh', 'mass_tot',
       'mass_tot_lh', 'mass_tot_nh', 'cut_area', 'cut_lh', 'cut_nh', 'cut_sum',
       'dr', 'bp', 'zp', 'ru', 'sg', 'ma', 'year_fe']].fillna(0)

wo['fb'] = wo['fb'].astype(np.uint8)
wo['fr'] = wo['fr'].astype(np.uint8)
wo['abt'] = wo['abt'].astype(np.uint16)
wo['uabt'] = wo['uabt'].astype(str)
wo['teilfl'] = wo['teilfl'].astype(np.uint8)

wo['admin'] = wo['admin'].astype(np.uint16)
wo['length'] = wo['length'].astype(np.float32)
wo['area'] = wo['area'].astype(np.float32)
wo['fly_date'] = wo['fly_date'].astype(str)
wo['year_fly'] = wo['year_fly'].astype(np.uint16)

wo['WO'] = wo['WO'].astype(str)

wo['fb_sap'] = wo['fb_sap'].astype(np.uint8)
wo['fr_sap'] = wo['fr_sap'].astype(np.uint8)
wo['abt_sap'] = wo['abt_sap'].astype(np.uint16)
wo['uabt_sap'] = wo['uabt_sap'].astype(str)
wo['teilfl_sap'] = wo['teilfl_sap'].astype(np.uint8)

wo['start_term'] = wo['start_term'].astype(str)
wo['uz'] = wo['uz'].astype(np.uint8)
wo['non_forest_type'] = wo['non_forest_type'].astype(np.uint8)
wo['economy'] = wo['economy'].astype(str)
wo['ww_sw'] = wo['ww_sw'].astype(str)
wo['sw_type'] = wo['sw_type'].astype(str)
wo['sea_level'] = wo['sea_level'].astype(np.uint16)
wo['exp'] = wo['exp'].astype(str)
wo['slope'] = wo['slope'].astype(np.uint8)
wo['site_type'] = wo['site_type'].astype(np.uint8)
wo['veg_type'] = wo['veg_type'].astype(str)
wo['growth_area'] = wo['growth_area'].astype(str)

wo['age'] = wo['age'].astype(np.uint16)
wo['BL'] = wo['BL'].astype(np.uint8)
wo['FI'] = wo['FI'].astype(np.uint8)
wo['LH'] = wo['LH'].astype(np.uint8)
wo['NH'] = wo['NH'].astype(np.uint8)
wo['LA'] = wo['LA'].astype(np.uint8)
wo['KI'] = wo['KI'].astype(np.uint8)
wo['DG'] = wo['DG'].astype(np.uint8)
wo['SK'] = wo['SK'].astype(np.uint8)

wo['mass_ha'] = wo['mass_ha'].astype(np.float32)
wo['mass_ha_lh'] = wo['mass_ha_lh'].astype(np.float32)
wo['mass_ha_nh'] = wo['mass_ha_nh'].astype(np.float32)
wo['mass_tot'] = wo['mass_tot'].astype(np.float32)
wo['mass_tot_lh'] = wo['mass_tot_lh'].astype(np.float32)
wo['mass_tot_nh'] = wo['mass_tot_nh'].astype(np.float32)

wo['cut_area'] = wo['cut_area'].astype(np.float32)
wo['cut_lh'] = wo['cut_lh'].astype(np.float32)
wo['cut_nh'] = wo['cut_nh'].astype(np.float32)
wo['cut_sum'] = wo['cut_sum'].astype(np.float32)

wo['dr'] = wo['dr'].astype(np.uint8)
wo['bp'] = wo['bp'].astype(np.uint8)
wo['zp'] = wo['zp'].astype(np.uint8)
wo['ru'] = wo['ru'].astype(np.uint8)
wo['sg'] = wo['sg'].astype(np.uint8)
wo['ma'] = wo['ma'].astype(str)
wo['year_fe'] = wo['year_fe'].astype(np.uint16)

In [54]:
wo

,fb,fr,abt,uabt,teilfl,admin,length,area,fly_date,year_fly,...,cut_lh,cut_nh,cut_sum,dr,bp,zp,ru,sg,ma,year_fe
0,171,1,65535,0,1,529,195.920105,1553.419067,2015-08-30,2015,...,0.0,0.0,0.0,0,0,0,0,0,0,0
1,171,1,65535,0,1,529,1125.586060,35217.652344,2015-08-30,2015,...,0.0,0.0,0.0,0,0,0,0,0,0,0
2,171,1,65535,0,1,529,946.503113,24184.699219,2015-08-30,2015,...,0.0,0.0,0.0,0,0,0,0,0,0,0
3,171,1,65535,0,1,529,319.914032,5266.038574,2015-08-30,2015,...,0.0,0.0,0.0,0,0,0,0,0,0,0
4,171,1,506,3,1,529,2402.546875,11624.655273,2015-08-30,2015,...,0.0,0.0,0.0,0,0,0,0,0,0,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354766,182,9,368,B,1,603,194.284134,1902.448975,2015-08-30,2015,...,0.0,0.0,0.0,0,0,0,0,0,0,2016
354767,182,9,368,B,1,603,740.912048,21281.767578,2015-08-30,2015,...,0.0,0.0,0.0,0,0,0,0,0,0,2016
354768,182,9,368,B,1,603,485.732819,10624.518555,2015-08-30,2015,...,0.0,0.0,0.0,0,0,0,0,0,0,2016
354769,182,9,368,B,2,603,2175.008789,64587.636719,2015-08-30,2015,...,0.0,0.0,0.0,0,0,0,0,0,0,2016


In [55]:
# calculate differenc between the fight (picture beeing taken) and fe (field work)
wo['age_diff'] = wo['year_fly'].astype(np.int16) - wo['year_fe'].astype(np.int16)

# set values to 0 if outside of valid renge
wo.loc[(wo['age_diff'] > 15) | (wo['age_diff'] < -15), 'age_diff'] = 0

In [56]:
wo.to_file('/home/philipp/Data/GIS/GIS_merged_SAP/2016/wo_2016.shp')

/tmp/ipykernel_8306/4018708524.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  wo.to_file('/home/philipp/Data/GIS/GIS_merged_SAP/2016/wo_2016.shp')
